此 .ipynb为按照行级，获得预测正确和错误的不同行级元素的贡献值分布。根据统计分析结果。
<br>打印出 <br>
[('returnstatement', 'return ( insertions . size () > 0 || ! unresolvedInsertions . isEmpty () || deletions . size () > 0 );', tensor(0.0590, dtype=torch.float64)), ('MethodDeclaration', 'public boolean areInsertionsOrDeletionsQueued () {', tensor(0.1059, dtype=torch.float64))]<br>[('行级语句类型','对应的行级语句','贡献值')]<br>
1<br>
索引<br>
根据上述的txt，挑出实例，不用挑太多

将每个实例中的类别贡献值与统计分析中的绝对值均值进行大小比较，大于绝对值均值的保留

In [70]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, TensorDataset


from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

# 解决服务器挂掉的问题
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [71]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device( "cpu")

In [72]:
MAX_LEN = 1024
MAX_EPOCHS = 10
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
NUM_CLASSES = 2
WEIGTH_DECAY = 1e-3

In [73]:
def retrieve_test_data():
    test_param = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/param/test.json")
    test_return = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/return/test.json")
    test_summary = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/summary/test.json")
    test_df = pd.concat([test_summary,test_param, test_return], axis=0)
    test_df = test_df[:20]
    test_df = test_df.reset_index(drop=True)
    return test_df
test_df = retrieve_test_data()

In [74]:
def get_lines_count(df):
    line_counts = []
    for i in range(len(df)):
        string = df.loc[i]['new_code_raw']
        line_count = len(string.split('\n'))
        line_counts.append(line_count)
    df['line_counts'] = line_counts
    return df
test_df = get_lines_count(test_df)
test_df.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,line_counts
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",6
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",4
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",28
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",4
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",16


In [75]:
df_clean = test_df
df_clean.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,line_counts
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",6
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",4
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",28
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",4
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",16


In [76]:
code_list = df_clean['new_code_raw']

In [77]:
code_list[0]

'  private static EmbeddedElasticsearchNode create(Settings settings) {\n    // ensure PainlessPlugin is installed or otherwise scripted fields would not work\n    Node node = new LocalNode(settings, Arrays.asList(Netty4Plugin.class, PainlessPlugin.class));\n    return new EmbeddedElasticsearchNode(node);\n  }\n'

In [78]:
df_clean.loc[0]['new_code_raw']

'  private static EmbeddedElasticsearchNode create(Settings settings) {\n    // ensure PainlessPlugin is installed or otherwise scripted fields would not work\n    Node node = new LocalNode(settings, Arrays.asList(Netty4Plugin.class, PainlessPlugin.class));\n    return new EmbeddedElasticsearchNode(node);\n  }\n'

In [79]:
len(df_clean)

20

In [80]:
# model = torch.load('save_GCBmodel.pt',map_location=torch.device('cuda:0'))
model = torch.load('D:/BERT_learing/CCDP/for_captum/save_model/save_GCBmodel.pt',map_location=torch.device('cpu'))

model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [81]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())  #输出为True，则安装无误


2.3.1+cpu
None
False


In [82]:
device

device(type='cpu')

In [83]:
tokenizer = RobertaTokenizer.from_pretrained("D:/BERT_learing/code_comment_inconsistency_detection/graphcodebert")

In [84]:
model.roberta.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(50265, 768, padding_idx=1)
  (position_embeddings): Embedding(514, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [85]:
model.roberta.encoder.layer[0]

RobertaLayer(
  (attention): RobertaAttention(
    (self): RobertaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): RobertaSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): RobertaIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): RobertaOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [86]:
input_embeddings = model.get_input_embeddings()

In [87]:
# predict和squad_pos_forward_func可以合成一个
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                  attention_mask=attention_mask )
    
    prediction = output.logits
    prediction_1 = nn.functional.softmax(prediction, dim=1)
    prediction = prediction_1.max(1).values
    out = torch.argmax(prediction_1, dim=-1)
    # prediction：每个输入样本的最大预测概率。
    # out：预测的类别标签。
    # prediction_1：所有类别的预测概率。    
    return prediction,out,prediction_1

In [88]:
def squad_pos_forward_func(inputs,position_ids=None, attention_mask=None, position=0):
    pred ,_,_= predict(inputs,
                     position_ids=position_ids,
                   attention_mask=attention_mask)
    return pred

In [89]:
ref_token_id = tokenizer.pad_token_id # 0
sep_token_id = tokenizer.sep_token_id # 101
cls_token_id = tokenizer.cls_token_id # 102
ref_token_id,sep_token_id,cls_token_id

(1, 2, 0)

In [90]:
# 注意长度
def truncate(ids,len_tru = 512):
    return ids[:len_tru] if len(ids) > len_tru else ids

In [91]:
# 这是单个数据的处理方式，应该要想数据集应该怎么处理
def construct_input_ref_pair(comment,AST_type,  ref_token_id, sep_token_id, cls_token_id):
    comment = tokenizer.encode(comment, add_special_tokens=False,truncation=True,max_length=512)
    AST_type = tokenizer.encode(AST_type, add_special_tokens=False,truncation=True,max_length=512)
    # construct input token ids
    input_ids = [cls_token_id] + comment + [sep_token_id] + AST_type + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(comment) + [sep_token_id] + \
        [ref_token_id] * len(AST_type) + [sep_token_id]
    input_ids = truncate(input_ids)
    ref_input_ids = truncate(ref_input_ids)
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(comment)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = model.roberta.embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = model.roberta.embeddings(ref_input_ids, token_type_ids=ref_token_type_ids, position_ids=ref_position_ids)
    
    return input_embeddings, ref_input_embeddings

In [92]:
def input_data_list(AST_list,comment_list):
    input_ids_all = []
    ref_input_ids_all = []
    position_ids_all = []
    attention_mask_all = []
    token_type_ids_all = []
    all_tokens_all = []
    for i in range(len(AST_list)):
        input_ids, ref_input_ids, comment_len = construct_input_ref_pair(comment_list[i],AST_list[i], ref_token_id,\
                                                                         sep_token_id, cls_token_id)
        token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, comment_len)
        position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
        attention_mask = construct_attention_mask(input_ids)
        
        indices = input_ids[0].detach().tolist()
        
        all_tokens = []                                       ###
        for _, token in enumerate(indices):
            all_tokens.append(tokenizer.decode([token]))
        
        input_ids_all.append(input_ids)
        ref_input_ids_all.append(ref_input_ids)
        position_ids_all.append(position_ids)
        attention_mask_all.append(attention_mask)
        token_type_ids_all.append(token_type_ids)
        all_tokens_all.append(all_tokens)

    return input_ids_all,ref_input_ids_all,position_ids_all,attention_mask_all,token_type_ids_all,all_tokens_all 

In [93]:
# 前k个贡献最高的word 和 token_type 和 position
# return value为归因贡献值  indices为词对应的索引  top_tokens为 词或位置或token_type
def get_topk_attributed_tokens(attrs,all_token_t, k=5):
    values_max, indices_max = torch.topk(attrs, k)
    top_tokens_max = [all_token_t[idx] for idx in indices_max]
    values_min, indices_min = torch.topk(attrs, k, largest=False)
    top_tokens_min = [all_token_t[idx] for idx in indices_min] 
    
    return top_tokens_max, values_max, indices_max,top_tokens_min,values_min,indices_min


In [94]:
import re
import string

def split_punctuation(s):
    # 使用正则表达式匹配连续的标点符号或者字母和标点符号之间的位置
    splits = re.finditer(r'(?<=\w)(?=[{}])|(?<=[{}])(?=\w)'.format(string.punctuation, string.punctuation), s)
    
    # 获取所有分割位置
    split_positions = [match.start() for match in splits]
    
    # 在分割位置插入空格
    for pos in reversed(split_positions):
        s = s[:pos] + ' ' + s[pos:]
        
    s = s.replace("< s >", "<s>")
    s = s.replace("</ s >", "</s>")
    return s


In [95]:
# 从 all_tokens 还原为 原单词 ，并且计算归因值
def get_restore_words(code,comment,input_ids,all_tokens,attribution_num):
    all_tokens_decode = tokenizer.decode(input_ids)
    len_all_tokens_decode = len(all_tokens_decode) + 4
    # 使用decode获得的序列，去掉分词之后的空格    例 ' a' -> 'a'
    all_tokens_clean = []
    for token in all_tokens:
        s_without_leading_space = token.lstrip()
        all_tokens_clean.append(s_without_leading_space)
#     print('all_tokens_clean:\n',all_tokens_clean)
#     print('all_tokens_clean:\n',len(all_tokens_clean))
    

    # 获得 code_comment_baseline
    code = tokenizer.encode(code, add_special_tokens=False,truncation=True,max_length=512)
    comment = tokenizer.encode(comment, add_special_tokens=False,truncation=True,max_length=512)
    code_decode = tokenizer.decode(code)
    comment_decode = tokenizer.decode(comment)
    
    code_comment_baseline = tokenizer.decode(tokenizer.cls_token_id) + ' '+ comment_decode \
                            + ' '+ tokenizer.decode(tokenizer.sep_token_id) + ' ' + code_decode \
                            + ' ' + tokenizer.decode(tokenizer.sep_token_id)
    
    code_comment_baseline = code_comment_baseline[:len_all_tokens_decode]
    code_comment_baseline = split_punctuation(code_comment_baseline)
    code_comment_baseline = code_comment_baseline.split()

#     print('code_comment_baseline:\n',code_comment_baseline)
#     print('code_comment_baseline_len:\n',len(code_comment_baseline))

    
    # 获得 相邻有几个token合并在一块的列表times  为了以后再计算attribute时求和
    times = []
    token_index = 0
    for code_comment in code_comment_baseline:
        temp = ''
        time = 0
        while temp != code_comment:
            temp = temp + all_tokens_clean[token_index]
            token_index = token_index + 1
            time = time + 1
        times.append(time)
#     print('times:\n',times)
    
    attribute_sum = []
    start = 0
    for time in times:
        end = start + time
        attribute = sum(attribution_num[start:end]) / time
        attribute_sum.append(attribute)
        start = end
#     print('attribute_sum:\n',attribute_sum)
    return code_comment_baseline ,attribute_sum 

In [96]:
def get_line_code_and_attribute(code,code_all_tokens,attributions_num):
    # 特殊符号前  加空格
    code = split_punctuation(code)
    
    # 把  行前空格去掉  便于单词与行之间匹配
    code = [line.lstrip() for line in code.splitlines()]
    code = '\n'.join(code)  
    code_lineList = code.split('\n')
    code_lineList = [' '.join(x.split()) for x in code_lineList]
    # 有空行，把空行去掉
    code_lineList = [item for item in code_lineList if item != '']
    
    attribute = []
    i = 0  
    for code_line in code_lineList:
        count = 0
        if i < len(code_all_tokens):
            temp = code_all_tokens[i]
            attr = attributions_num[i]
            i = i + 1
        while((i < len(code_all_tokens)) and(temp != code_line)):
            attr = attr + attributions_num[i]
            temp = temp + ' ' + code_all_tokens[i]
            i = i + 1
            count = count + 1
        attribute.append(attr/count)

    return code_lineList,attribute   

In [97]:
def remove_before_and_including(lst, element):
    if element in lst:
        index = lst.index(element)
        lst_c = lst[index+1:]
        if element in lst_c:
            lst_c.remove(element)
        return lst_c
    else:
        return lst
    
def remove_after_including(lst, element):
    if element in lst:
        index = lst.index(element)
        return lst[:index + 1]
    else:
        return lst

In [98]:
def final_all_attribute(code,all_tokens,attributions_num):
    # 删除 all_token 列表中的<s>注释</s>  </s>
    code_all_tokens = remove_before_and_including(all_tokens,'</s>')
    comment_all_tokens = remove_after_including(all_tokens,'</s>')  
    index = all_tokens.index('</s>')
    attribute_comment = attributions_num[:index+1]
#     print(attribute_comment)
    
    attribute_code = attributions_num[index+1:-1]
#     print(attribute_code,len(attribute_code))
    code_lineList_token,attribute_num_code = get_line_code_and_attribute(code,code_all_tokens,attribute_code)
##     print(attribute_num_code)
    attribute_num_code = torch.stack(attribute_num_code)
    
    new_all_tokens = comment_all_tokens + code_lineList_token
    attributions_num_all = torch.cat((attribute_comment, attribute_num_code))

#     attributions_num_all = attribute_comment + attribute_num_code
#     print(len(attribute_comment),len(attribute_num_code))
#     print(new_all_tokens)
    return new_all_tokens,attributions_num_all,code_lineList_token,attribute_num_code

In [99]:
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

lig = LayerIntegratedGradients(squad_pos_forward_func,input_embeddings)

In [100]:
def interpret_sentence_2(code,comment,old_code,input_ids,ref_input_ids, token_type_ids,\
                         position_ids, attention_mask, all_tokens):
    pre ,out,_ = predict(input_ids,  position_ids=position_ids,attention_mask=attention_mask)
  
    attributions_ig, delta_ig = lig.attribute(input_ids, baselines=ref_input_ids,\
                           additional_forward_args=(position_ids,attention_mask,0),return_convergence_delta=True,internal_batch_size=8)
    
    attributions = attributions_ig.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
#     print(delta_ig)
#     print(attributions)
    try:
        all_tokens ,attributions = get_restore_words(code,comment,input_ids[0],all_tokens,attributions)     # 合并为一个单词
        attributions = torch.tensor(attributions)
#         print(all_tokens)


        # code_lineList_token 和 attribute_num_code 用于后续统计分析
        new_all_tokens,attributions_num_all,code_lineList_token,attribute_num_code = final_all_attribute(code,all_tokens,attributions)

        return out,code_lineList_token,attribute_num_code
    except Exception as e:
#         pass
        print("解析错误")
        return -1,_,_

判断行级贡献值分布情况，将一些特殊语句变为一个具有代表性的词

In [101]:
# 单个示例获取各种statement 的函数
def get_ifstatement(code_line,attribution_num):
    pattern = re.compile(r'\bif\b')    
    code_contains_if = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)]   
    attr_contains_if = [attribution_num[code_contains_if[i][0]] for i in range(len(code_contains_if))]
    merged_list = [('ifstatement', s, attr) for (_, s), attr in zip(code_contains_if, attr_contains_if)]
    return merged_list
def get_forstatement(code_line,attribution_num):
    pattern = re.compile(r'\bfor\b')    
    code_contains_for = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)]   
    attr_contains_for = [attribution_num[code_contains_for[i][0]] for i in range(len(code_contains_for))]
    merged_list = [('forstatement', s, attr) for (_, s), attr in zip(code_contains_for, attr_contains_for)]
    return merged_list
def get_whilestatement(code_line,attribution_num):
    pattern = re.compile(r'\bwhile\b')    
    code_contains_while = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)]   
    attr_contains_while = [attribution_num[code_contains_while[i][0]] for i in range(len(code_contains_while))]
    merged_list = [('whilestatement', s, attr) for (_, s), attr in zip(code_contains_while, attr_contains_while)]
    return merged_list
def get_variableDeclaration(code_line,attribution_num):
    pattern = re.compile(r'\bnew\b')    
    code_contains_new = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)]       
    attr_contains_new = [attribution_num[code_contains_new[i][0]] for i in range(len(code_contains_new))]
    merged_list = [('variableDeclaration', s, attr) for (_, s), attr in zip(code_contains_new, attr_contains_new)]
    return merged_list
def get_expression(code_line,attribution_num):
    pattern = re.compile(r'\b=\b')    
    code_contains_exp = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)] 
    attr_contains_exp = [attribution_num[code_contains_exp[i][0]] for i in range(len(code_contains_exp))]
    merged_list = [('expression', s, attr) for (_, s), attr in zip(code_contains_exp, attr_contains_exp)]
    return merged_list
def get_try_catch(code_line,attribution_num):
    pattern = re.compile(r'\btry\b')    
    code_contains_try_catch = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)] 
    attr_contains_try_catch = [attribution_num[code_contains_try_catch[i][0]] for i in range(len(code_contains_try_catch))]
    merged_list = [('trystatement', s, attr) for (_, s), attr in zip(code_contains_try_catch, attr_contains_try_catch)]
    return merged_list
def get_return(code_line,attribution_num):
    pattern = re.compile(r'\breturn\b')    
    code_contains_return = [(i, s) for i, s in enumerate(code_line) if pattern.search(s)]
    attr_contains_return = [attribution_num[code_contains_return[i][0]] for i in range(len(code_contains_return))]
    merged_list = [('returnstatement', s, attr) for (_, s), attr in zip(code_contains_return, attr_contains_return)]
    return merged_list
def get_MethodDeclaration(code_line,attribution_num):
    code_contains_Method = [code_line[0]]
    attr_contains_Method = [attribution_num[0]]
    merged_list = list(zip(['MethodDeclaration'],code_line, attribution_num))
    return merged_list

In [102]:
# 集成到数据集中 , 对返回值进行数据分析 , 六个列表试着画小提琴图
from itertools import chain

def get_all_statement(code_line_list,attribution_num_list):
    type_code_attr = []
    for (code_line,attribution_num) in zip(code_line_list,attribution_num_list):
#         print(code_line,attribution_num)
        merged_list_if = get_ifstatement(code_line,attribution_num)
        merged_list_for = get_forstatement(code_line,attribution_num)
        merged_list_while = get_whilestatement(code_line,attribution_num)
        merged_list_new = get_variableDeclaration(code_line,attribution_num)
        merged_listexp = get_expression(code_line,attribution_num)
        merged_list_try_catch = get_try_catch(code_line,attribution_num)
        merged_list_return = get_return(code_line,attribution_num)
        merged_list_Method = get_MethodDeclaration(code_line,attribution_num)
        
        conditions = [
            merged_list_if,
            merged_list_for,
            merged_list_while,
            merged_list_new,
            merged_listexp,
            merged_list_try_catch,
            merged_list_return ,
            merged_list_Method
        ]
        # 合并上述非空的列表
        merged_conditions = list(chain(*[lst for lst in conditions if lst]))
        type_code_attr.append(merged_conditions)
        
    return type_code_attr

In [103]:
def analyse_line_attribution(df):
    code_list = df['new_code_raw']
    comment_list = df['old_comment_raw']
    code_list_old = df['old_code_raw']
    ground_lable = df['label']
    
    input_ids_all,ref_input_ids_all,position_ids_all,attention_mask_all,\
    token_type_ids_all,all_tokens_all= input_data_list(code_list,comment_list)
    
    code_lineList_token_true = []
    attribute_num_code_true = []
    index_true = []
    code_lineList_token_false = []
    attribute_num_code_false = []
    index_false = []

    for i in range(len(code_list)): 
        out,code_lineList_token,attribute_num_code \
        = interpret_sentence_2(code_list[i],comment_list[i],\
                               code_list_old[i],input_ids_all[i],\
                               ref_input_ids_all[i], token_type_ids_all[i],\
                               position_ids_all[i],attention_mask_all[i], all_tokens_all[i])
        if out != -1:
            if out == ground_lable[i]:
                index_true.append(i)
                code_lineList_token_true.append(code_lineList_token)
                attribute_num_code_true.append(attribute_num_code)
            else :
                index_false.append(i)
                code_lineList_token_false.append(code_lineList_token)
                attribute_num_code_false.append(attribute_num_code)
    return index_true,index_false,code_lineList_token_true,attribute_num_code_true,code_lineList_token_false,attribute_num_code_false

In [104]:
index_true,index_false,code_lineList_token_true,attribute_num_code_true,code_lineList_token_false,attribute_num_code_false\
= analyse_line_attribution(df_clean)

解析错误
解析错误
解析错误
解析错误


In [105]:
len(index_true),len(code_lineList_token_true),len(attribute_num_code_true)

(9, 9, 9)

In [106]:
# average_list 为 均值的元组 (类型名，均值)
def get_filter_pair(list_pair,average_list,index_all):

    type_dict = dict(average_list)
    
    # 过滤符合条件的子列表，并记录被删除的索引
    filtered_nested_statements = []
    removed_indices = []
    
    for index, sublist in enumerate(list_pair):
        if all(stmt_type in type_dict and value > type_dict[stmt_type] for stmt_type, _, value in sublist if stmt_type in type_dict):
            filtered_nested_statements.append(sublist)
        else:
            removed_indices.append(index)
    index_all_t = [index_all[i] for i in range(len(index_all)) if i not in removed_indices]
  
    return filtered_nested_statements,index_all_t

true

In [107]:
type_code_attr_true= get_all_statement(code_lineList_token_true,attribute_num_code_true)

In [108]:
len(index_true),len(type_code_attr_true)

(9, 9)

In [109]:
# 由均值而来 iqr
type_list_true = [
    ('MethodDeclaration',0.143),
    ('variableDeclaration',0.087),
    ('trystatement',0.102),
    ('returnstatement',0.129)
]

In [109]:
# 由均值而来 Winsorization 
type_list_true_1 = [
    ('MethodDeclaration',0.182),
    ('variableDeclaration',0.09),
    ('trystatement',0.100),
    ('returnstatement',0.144)
]

In [ ]:
filtered_statements_true,filter_true_index = get_filter_pair(type_code_attr_true, type_list_true,index_true)

In [110]:
filter_true_index

[1, 3, 7, 13]

In [111]:
filtered_statements_true

[[('returnstatement',
   'return ( insertions . size () > 0 || ! unresolvedInsertions . isEmpty () || deletions . size () > 0 );',
   tensor(0.0590, dtype=torch.float64)),
  ('MethodDeclaration',
   'public boolean areInsertionsOrDeletionsQueued () {',
   tensor(0.1059, dtype=torch.float64))],
 [('returnstatement',
   'return Lists . transform ( type . getFieldList (), GET _ TYPE );',
   tensor(0.0478, dtype=torch.float64)),
  ('MethodDeclaration',
   'public static List < RelDataType > getFieldTypeList ( final RelDataType type ) {',
   tensor(0.0580, dtype=torch.float64))],
 [('ifstatement',
   'if ( txtPattern == null ) {',
   tensor(0.0214, dtype=torch.float64)),
  ('variableDeclaration',
   'txtPattern = new ZapTextField ();',
   tensor(0.0512, dtype=torch.float64)),
  ('returnstatement',
   'return txtPattern ;',
   tensor(0.0719, dtype=torch.float64)),
  ('MethodDeclaration',
   'public ZapTextField getTxtPattern () {',
   tensor(0.0075, dtype=torch.float64))],
 [('returnstatemen

In [112]:
# 打开一个txt文件用于写入
with open("linetype_attribution_index_true.txt", "w") as file:
    max_length = max(len(filtered_statements_true), len(filter_true_index))
    
    for i in range(max_length):
        # 如果当前索引在第一个列表的范围内，写入奇数行
        if i < len(filtered_statements_true):
            file.write(str(filtered_statements_true[i]) + "\n")
        # 如果当前索引在第二个列表的范围内，写入偶数行
        if i < len(filter_true_index):
            file.write(str(filter_true_index[i]) + "\n")


false

In [113]:
type_code_attr_false= get_all_statement(code_lineList_token_false,attribute_num_code_false)

In [114]:
len(index_false),len(type_code_attr_false)

(7, 7)

In [115]:
# 由均值而来 iqr
type_list_false = [
    ('MethodDeclaration',0.176),
    ('whilestatement',0.100),
    ('trystatement',0.102),
    ('returnstatement',0.139)
]

In [115]:
# 由均值而来 Winsorization 
type_list_false_1 = [
    ('MethodDeclaration',0.21),
    ('whilestatement',0.106),
    ('trystatement',0.094),
    ('returnstatement',0.159)
]

In [ ]:
filtered_statements_false,filter_false_index = get_filter_pair(type_code_attr_false, type_list_false,index_false)

In [117]:
filter_false_index

[5, 6, 14]

In [116]:
with open("linetype_attribution_index_false.txt", "w") as file:
    max_length = max(len(filtered_statements_false), len(filter_false_index))
    
    for i in range(max_length):
        # 如果当前索引在第一个列表的范围内，写入奇数行
        if i < len(filtered_statements_false):
            file.write(str(filtered_statements_false[i]) + "\n")
        # 如果当前索引在第二个列表的范围内，写入偶数行
        if i < len(filter_false_index):
            file.write(str(filter_false_index[i]) + "\n")
